<a href="https://colab.research.google.com/github/Junaeid-Shoaib/AI_Agents/blob/main/Blog_writer_using_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# To install crewAI, you need to have Python >=3.10 and <=3.13 installed on your system:
import sys
print(sys.version)

3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]


In [ ]:
!pip install crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.2/379.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.8 MB/s eta 0

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Importing Libraries
Installing crewAi library and crewAI tools

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('secretKey')  #For LLM Open-ai key

* role: agent's function within the crew
* goal: agent's objective. Guides agent's decision making process
* backstory: provides context to agent's role and goal. Helps in interation and collaboration.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic",
    allow_delegation=False,
    verbose=True
)

# Agent: Writer

In [ ]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

#Agent: Editor

In [ ]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

# Define the Tasks
Tasks are specific assignments, completed by agents. They provide all necessary details for execution, such as a description, expected_output , agent responsible, tools, etc.

description: Tells about what task entails
expected_output: Tells about what task output should look like
agent: Agent responsible for the task

# Task: Plan

In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

# Task: Write

In [ ]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "In starting each section should have 2 or 3 paragraphs and"
        "in the middle section should have 4 or 5 paragraphs and"
        "in the third or final section should have 2 or 3 paragraphs.",
    agent=writer
)

# Task: Edit

In [ ]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "In starting each section should have 2 or 3 paragraphs and"
                    "in the middle section should have 4 or 5 paragraphs and"
                    "in the third or final section should have 2 or 3 paragraphs.",
    agent=editor
)

# Forming the crew
Crew is group of AI agents, collaborating to perform set of tasks.

tasks: list of tasks
agents: list of agents
process (optional): work flow (e.g., sequential, hierarchical) default : sequential
verbose=2 allows you to see all the logs of the execution.

In [ ]:
crew = Crew(
    tasks=[plan, write, edit],
    agents=[planner, writer, editor],
    verbose=False
)

# Kick It Off

In [ ]:
topic = "Artificial Intelligence"
result = crew.kickoff(inputs={"topic": topic})

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.            │
│  2. Identify the target audience, considering their interests and pain points.                                  │
│  3. Develop a detailed content outline including an introduction, key points, and a call to action.             │
│  4. Include SEO keywords and relevant data or sources.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Content Plan Document on Artificial Intelligence**                                                           │
│                                                                                                                 │
│  **1. Latest Trends, Key Players, and Noteworthy News in Artificial Intelligence:**                             │
│     - **Trends:**                                                                                               │
│       - Increased adoption of AI in healthcare for diagnostics and treatment plans.                             │
│       - Advancements in natural language processing (NLP) with models like ChatGPT and BERT.                    │
│       - Growth of AI in cybersecurity for threat detection and mitigation.                                      │
│       - The rise of generative AI tools for creative tasks such as content generation and art creation.         │
│       - Use of AI in automating supply chain and logistics operations.                                          │
│     - **Key Players:**                                                                                          │
│       - Google (DeepMind, TensorFlow)                                                                           │
│       - Microsoft (Azure AI, Microsoft Cognitive Services)                                                      │
│       - OpenAI (ChatGPT, DALL-E)                                                                                │
│       - IBM (Watson)                                                                                            │
│       - Amazon (AWS AI services)                                                                                │
│     - **Noteworthy News:**                                                                                      │
│       - Recent breakthroughs in AI ethics and governance discussions.                                           │
│       - Legislative developments affecting AI regulation, such as the European Union’s AI Act.                  │
│       - Major funding rounds for AI startups and emerging technologies.                                         │
│                                                                                                                 │
│  **2. Target Audience:**                                                                                        │
│     - **Demographics:**                                                                                         │
│       - Tech enthusiasts and professionals (ages 25-45)                                                         │
│       - Business leaders (CEOs, CTOs, and decision-makers in various industries)                                │
│       - Students and researchers in AI and related fields                                                       │
│     - **Interests:**                                                                                            │
│       - The latest innovations in AI technology and applications                                                │
│       - Understanding of how AI can improve operational efficiency                                              │
│       - Ethical implications and governance in AI use                                                           │
│     - **Pain Points:**                                 

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Writer                                                                                          │
│                                                                                                                 │
│  Task: 1. Use the content plan to craft a compelling blog post on Artificial Intelligence.                      │
│  2. Incorporate SEO keywords naturally.                                                                         │
│  3. Sections/Subtitles are properly named in an engaging manner.                                                │
│  4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing             │
│  conclusion.                                                                                                    │
│  5. Proofread for grammatical errors and alignment with the brand's voice.                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Writer                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # The Future of Artificial Intelligence: Trends, Insights, and Innovations                                     │
│                                                                                                                 │
│  ## Introduction                                                                                                │
│                                                                                                                 │
│  Artificial Intelligence (AI) has evolved dramatically over the past few years, transitioning from a niche      │
│  field to a cornerstone of technological advancement across various sectors. Its significance can't be          │
│  overstated as it redefines operational norms in healthcare, cybersecurity, logistics, and creative             │
│  industries. As we step further into the 21st century, the importance of staying informed about the latest      │
│  trends and developments in AI becomes ever more crucial.                                                       │
│                                                                                                                 │
│  This blog post highlights the current trends shaping the AI landscape, key players driving innovation, and     │
│  the ethical considerations surrounding this transformative technology. Through this exploration, we will       │
│  uncover how AI is not just reshaping industries but is also posing new challenges and considerations for       │
│  businesses and society at large.                                                                               │
│                                                                                                                 │
│  ## Current Trends in AI                                                                                        │
│                                                                                                                 │
│  One of the most compelling trends we are witnessing is the increased adoption of AI in healthcare. Advances    │
│  in machine learning and data analysis mean that AI can assist in diagnostics and formulate more accurate       │
│  treatment plans. Companies are leveraging AI to analyze vast datasets, enabling quicker and better-informed    │
│  medical decisions. A notable example is the integration of AI in radiology for diagnosing illnesses like       │
│  pneumonia or cancer, where algorithms surpass traditional methods in accuracy and speed.                       │
│                                                                                                                 │
│  In the realm of natural language processing (NLP), advancements such as OpenAI's ChatGPT and Google's BERT     │
│  have taken center stage. These models allow for more sophisticated interactions between humans and machines,   │
│  democratizing the power of AI. Businesses are using NLP for everything from automating customer service        │
│  inquiries to enhancing content creation strategies, showcasing how AI can streamline operations and enhance    │
│  productivity.                                                                                                  │
│                                                        

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # The Future of Artificial Intelligence: Trends, Insights, and Innovations                                     │
│                                                                                                                 │
│  ## Introduction                                                                                                │
│                                                                                                                 │
│  Artificial Intelligence (AI) has evolved dramatically over the past few years, transitioning from a niche      │
│  field to a cornerstone of technological advancement across various sectors. Its significance cannot be         │
│  overstated as it redefines operational norms in healthcare, cybersecurity, logistics, and creative             │
│  industries. As we step further into the 21st century, the importance of staying informed about the latest      │
│  trends and developments in AI is more crucial than ever.                                                       │
│                                                                                                                 │
│  This blog post highlights the current trends shaping the AI landscape, key players driving innovation, and     │
│  the ethical considerations surrounding this transformative technology. Through this exploration, we will       │
│  uncover how AI is not just reshaping industries but is also posing new challenges and considerations for       │
│  businesses and society at large.                                                                               │
│                                                                                                                 │
│  ## Current Trends in AI                                                                                        │
│                                                                                                                 │
│  One of the most compelling trends we are witnessing is the increased adoption of AI in healthcare. Advances    │
│  in machine learning and data analysis allow AI to assist in diagnostics and to formulate more accurate         │
│  treatment plans. Companies are leveraging AI to analyze vast datasets, enabling quicker and better-informed    │
│  medical decisions. A notable example is the integration of AI in radiology for diagnosing illnesses like       │
│  pneumonia or cancer, where algorithms surpass traditional methods in accuracy and speed.                       │
│                                                                                                                 │
│  In the realm of natural language processing (NLP), advancements such as OpenAI's ChatGPT and Google's BERT     │
│  have taken center stage. These models facilitate more sophisticated interactions between humans and machines,  │
│  democratizing the power of AI. Businesses use NLP for everything from automating customer service inquiries    │
│  to enhancing content creation strategies, showcasing how AI can streamline operations and enhance              │
│  productivity.                                                                                                  │
│                                                        

In [ ]:
from IPython.display import Markdown

# Extract markdown content from between the code blocks
markdown_content = result.raw.split('```markdown\n')[1].split('\n```')[0]
Markdown(markdown_content)

# The Future of Artificial Intelligence: Trends, Insights, and Innovations

## Introduction

Artificial Intelligence (AI) has evolved dramatically over the past few years, transitioning from a niche field to a cornerstone of technological advancement across various sectors. Its significance cannot be overstated as it redefines operational norms in healthcare, cybersecurity, logistics, and creative industries. As we step further into the 21st century, the importance of staying informed about the latest trends and developments in AI is more crucial than ever.

This blog post highlights the current trends shaping the AI landscape, key players driving innovation, and the ethical considerations surrounding this transformative technology. Through this exploration, we will uncover how AI is not just reshaping industries but is also posing new challenges and considerations for businesses and society at large.

## Current Trends in AI

One of the most compelling trends we are witnessing is the increased adoption of AI in healthcare. Advances in machine learning and data analysis allow AI to assist in diagnostics and to formulate more accurate treatment plans. Companies are leveraging AI to analyze vast datasets, enabling quicker and better-informed medical decisions. A notable example is the integration of AI in radiology for diagnosing illnesses like pneumonia or cancer, where algorithms surpass traditional methods in accuracy and speed.

In the realm of natural language processing (NLP), advancements such as OpenAI's ChatGPT and Google's BERT have taken center stage. These models facilitate more sophisticated interactions between humans and machines, democratizing the power of AI. Businesses use NLP for everything from automating customer service inquiries to enhancing content creation strategies, showcasing how AI can streamline operations and enhance productivity.

Cybersecurity is another field experiencing rapid transformation due to AI's capabilities. With cyber threats becoming more intricate and frequent, AI models can analyze patterns and predict potential security breaches faster than human experts. Tools that learn and adapt in real-time are essential in staying one step ahead of cybercriminals, highlighting AI’s critical role in defending digital infrastructures.

The rise of generative AI tools is a noteworthy trend that cannot be overlooked. Artists, writers, and content creators are discovering new avenues of expression through tools that can conjure original art and literature. This intersection of technology and creativity raises fascinating questions about the future role of human creators in an era where machines can perform similar tasks.

Furthermore, AI is automating supply chains and logistics operations, providing businesses with enhanced efficiency. By harnessing AI-driven insights, companies can predict demand fluctuations and optimize inventory levels. This transformation has proven essential, especially in a global economy where agility and adaptability are paramount.

## Key Players in AI Development

Understanding who drives the AI landscape is as important as recognizing its trends. Key players such as Google, Microsoft, OpenAI, IBM, and Amazon have made significant contributions that shape our engagement with AI technology.

Google has been at the forefront with its DeepMind and TensorFlow initiatives, pushing the boundaries of what AI can achieve. DeepMind's successes in mastering complex games and developing AI that aids in healthcare demonstrate the transformative implications of their work. TensorFlow, an open-source machine learning library, enables developers globally, fostering a flourishing ecosystem around AI research and applications.

Microsoft's Azure AI platform and Cognitive Services provide businesses essential tools for integrating AI into their operations. Their focus on ethical AI and responsible innovation is setting a precedent in the industry, encouraging businesses to prioritize ethical considerations in their AI deployments.

OpenAI's initiatives, particularly with ChatGPT and DALL-E, aim to widen accessibility to AI resources and foster innovation. Their commitment to ensuring that AI technologies are safely and responsibly developed impacts current practices and future regulations.

IBM's Watson continues to lead the charge in intelligent enterprise solutions, showcasing AI's remarkable capabilities to enhance productivity and decision-making in various sectors. Similarly, Amazon's suite of AWS AI services offers versatile tools for companies to integrate AI without the overhead of extensive infrastructure.

## Impact of AI on Businesses

As businesses grapple with the complexities of AI implementation, understanding its potential to drive efficiency is essential. AI is reshaping workflows in remarkable ways—organizations can leverage predictive analytics to streamline operations, resulting in improved decision-making and cost reductions.

The challenges are tangible: navigating the data privacy landscape, addressing ethical concerns, and adapting organizational culture to embrace AI technology. While some fear job displacement due to automation, AI also creates opportunities for new roles and skill development. By focusing on reskilling and upskilling employees, businesses can prepare their workforce for the future.

Effective AI integration requires strategic planning and an investment in comprehensive training for staff to ensure successful implementation. Companies must remain agile, refining processes to stay aligned with changing technologies and consumer behaviors. Collaboration between various stakeholders—including tech experts, business leaders, and ethical boards—forms the foundation for leveraging AI responsibly.

However, deploying AI solutions must go hand in hand with transparency and ethical considerations. Balancing the benefits of speed and efficiency against the responsibilities of protecting user data and mitigating bias is no small feat. Addressing these concerns will require robust frameworks and adherence to regulatory guidelines as discussions escalate around the ethical use of AI.

## Ethics and Regulation in AI

The conversation surrounding AI is increasingly focusing on its ethical implications and the urgent need for regulation. As AI systems are integrated into daily life, concerns such as data privacy, algorithmic bias, and accountability come to the forefront. Policymakers are increasingly aware of the importance of establishing regulations to govern AI use effectively.

Recent developments, such as the European Union’s AI Act, signal a larger movement toward ethical governance in AI. This legal framework aims to create secure and trustworthy AI systems while encouraging innovation. Such regulations could set a global standard, prompting other nations to follow suit in addressing the complex ethical questions posed by AI technologies.

Ongoing governance discussions are essential in cultivating confidence among users and stakeholders. By promoting transparency and ethics in AI deployment, businesses can navigate the intricacies of this technology while assuring users of the integrity behind decision-making processes.

## Future Outlook for AI

Looking ahead, the future of AI is saturated with potential advancements and innovative applications. From enhancing personalized experiences in education and healthcare to revolutionizing research methodologies in various fields, the scope of AI's impact is vast. Continued investment in AI research, along with collaborative efforts among leaders in technology, policy, and ethics, will ensure that AI develops in a way that benefits society.

Emerging applications, particularly in areas like environmental sustainability and disaster response, could leverage AI to provide solutions to pressing global challenges. As AI tools become more advanced, they may help uncover insights that can aid in combating issues like climate change, offering a framework for a more sustainable future.

For businesses and decision-makers, maintaining an open dialogue about AI investments and embracing a forward-thinking approach is vital. By staying informed and engaged with these trends, industries can position themselves at the forefront of this transformative technology.

## Conclusion

Artificial Intelligence is on an upward trajectory, ushering in innovative changes across multiple sectors. By understanding the latest trends, key players, and ethical considerations surrounding AI, businesses and individuals can navigate this complex landscape more effectively.

As we immerse ourselves in the AI revolution, it is essential to remain vigilant about the impact of our technological choices. With the right approach to investment, governance, and ethical frameworks, we can harness AI’s potential to foster a more efficient and equitable future.

**Call to Action:** Stay tuned for our continued exploration of AI trends and innovations! Subscribe for updates and join us in our upcoming webinars and forums to engage directly with the latest in AI technology.